In [2]:
import pandas as pd
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
%matplotlib inline
import codecs
import re
import enchant
import re
import nltk

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, Activation
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder

Using Theano backend.


In [4]:
filename = "./Data/data_segments2.json"
data = open(filename).read()

In [5]:
def load_json_multiple(segments):
    chunk = ""
    for segment in segments:
        chunk += segment
        try:
            yield json.loads(chunk)
            chunk = ""
        except ValueError:
            pass

In [6]:
data = []
with open(filename) as f:
    for parsed_json in load_json_multiple(f):
        data.append(parsed_json)
print len(data)

113


In [7]:
data1 = []
for i in xrange(len(data)):
    for j in data[i]["transcription_merged"]:
        temp = j
        temp["_id"] = data[i]["_id"]
        data1.append(temp)
df = pd.DataFrame(data1)
df.head()

,_id,conv_no,end_time,line,question,segment,speaker,start_time
0,d7da99f7-013b-4d63-84d9-421226ee5249,0.0,30,at indian east india company now by the way wh...,NaN,NaN,Rep,0
1,d7da99f7-013b-4d63-84d9-421226ee5249,1.0,31,HMM.,NaN,NaN,Prospect,30
2,d7da99f7-013b-4d63-84d9-421226ee5249,2.0,34,so you're creating tons of synonyms on the bac...,NaN,NaN,Rep,31
3,d7da99f7-013b-4d63-84d9-421226ee5249,3.0,35,it is manual.,NaN,NaN,Prospect,34
4,d7da99f7-013b-4d63-84d9-421226ee5249,4.0,53,right exactly you got it then next comes is th...,NaN,NaN,Rep,35


In [8]:
df1 = df[~df.segment.isnull()]
df1["predicted_value"] = np.where(df1["segment"]=="Next steps and action items",1,0)
df1 = df1.sample(frac=1).reset_index(drop=True)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
df1.end_time = df1.end_time.astype(np.float64)
df1.start_time = df1.start_time.astype(np.float64)
df1.conv_no = df1.conv_no.astype(np.float64)

In [10]:
df1 = df1.set_index(np.arange(len(df1)))

In [11]:
df1["time_difference"] = df1["end_time"] - df1["start_time"]
df1["time_difference"] = df1["time_difference"]/max(df1["time_difference"])
df1["conv_no"] = df1["conv_no"] / max(df1["conv_no"])

In [12]:
le = LabelEncoder()
df1["speaker"] = le.fit_transform(df1["speaker"])

In [13]:
df1 = df1.drop(["_id","question"],axis=1)

In [14]:
df1["line"] = df1["line"].apply(lambda x: re.sub(r"[^\x00-\x7f]"," ",x))
df1["line"] = df1["line"].apply(lambda x: re.sub(r"^[a-zA-Z0-9\s]", "", x))
df1["line"] = df1["line"].apply(str)

In [15]:
df1["line"] = df1["line"].apply(lambda x: re.sub(r"[\s]+"," ", x).strip())
df1["line"] = df1["line"].apply(lambda x: x.lower())

In [16]:
max_features = 5000
tokenizer = Tokenizer(nb_words=max_features, split=' ')
tokenizer.fit_on_texts(df1['line'].values)

In [17]:
X = tokenizer.texts_to_sequences(df1['line'].values)
X = pad_sequences(X)

In [18]:
embed_dim = 256
lstm_out = 512

In [19]:
type(X)

numpy.ndarray

In [20]:
model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length=X.shape[1], dropout=0.2))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2, return_sequences=True))
model.add(GRU(lstm_out, activation='relu'))
model.add(Dense(lstm_out, input_dim=lstm_out, activation='tanh'))
model.add(Dense(lstm_out, input_dim=lstm_out, activation='relu'))
model.add(Dense(lstm_out, input_dim=lstm_out, activation='sigmoid'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 1268, 256)     1280000     embedding_input_1[0][0]          
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 1268, 512)     1574912     embedding_1[0][0]                
____________________________________________________________________________________________________
gru_1 (GRU)                      (None, 512)           1574400     lstm_1[0][0]                     
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 512)           262656      gru_1[0][0]                      
___________________________________________________________________________________________

In [21]:
length = int(len(df1)*.7)
print length

3228


In [22]:
y_train = df1["predicted_value"].iloc[0:length]
y_train = pd.get_dummies(y_train).values

y_test =  df1["predicted_value"].iloc[length:]
y_test = pd.get_dummies(y_test).values

In [23]:
x_train = X[:length]
x_test = X[length:]

In [24]:
batch_size = 64
model.fit(x_train, y_train, nb_epoch=2, batch_size=batch_size, verbose=2)
#Y = model.predict_proba(X)

Epoch 1/2


MemoryError: alloc failed
Apply node that caused the error: Alloc(TensorConstant{(1, 1, 1) of 0.0}, TensorConstant{1268}, Elemwise{Composite{((i0 * i1) // i2)}}.0, TensorConstant{2048})
Toposort index: 149
Inputs types: [TensorType(float32, (True, True, True)), TensorType(int64, scalar), TensorType(int64, scalar), TensorType(int64, scalar)]
Inputs shapes: [(1, 1, 1), (), (), ()]
Inputs strides: [(4, 4, 4), (), (), ()]
Inputs values: [array([[[ 0.]]], dtype=float32), array(1268), array(64), array(2048)]
Outputs clients: [[IncSubtensor{Inc;:int64:}(Alloc.0, Subtensor{::int64}.0, Constant{1268}), IncSubtensor{InplaceInc;int64::}(Alloc.0, IncSubtensor{Inc;:int64:}.0, Constant{0}), forall_inplace,cpu,grad_of_scan_fn}(TensorConstant{1268}, Elemwise{mul,no_inplace}.0, Elemwise{mul,no_inplace}.0, Elemwise{tanh}.0, Elemwise{mul,no_inplace}.0, Alloc.0, Elemwise{mul,no_inplace}.0, Elemwise{Composite{(i0 - sqr(i1))}}.0, Subtensor{int64:int64:int64}.0, Subtensor{int64:int64:int64}.0, Elemwise{Add}[